In [ ]:
from train import train_gan
from dataclass.MicroImageClass import MicroImages
import torch
from matplotlib import pyplot as plt
from dataclass.prepare_data import prepare_datasets
from dataclass.dataset import ImagesDataset, convert_image
import torchvision
import numpy as np
from torchvision.utils import save_image
from models.Generator import GeneratorV0
from ignite.metrics import SSIM, PSNR
from load_models import load_gen_02_05_2021, load_gen_03_05_2021, load_gen_04_05_2021, load_gen_06_05_2021, load_gen_05_05_2021, load_gen_mse_ssim, load_gen_mse_lpips

In [ ]:
models_4_0_1 = [load_gen_mse_ssim(), load_gen_mse_lpips()]
names_4_0_1 = ["mse-ssim", "mse-lpips"]

models_4_1_1 = [load_gen_06_05_2021(), load_gen_05_05_2021()]
names_4_1_1 = ["06", "05"]

models_2 = [load_gen_02_05_2021(), load_gen_03_05_2021(), load_gen_04_05_2021()]
names_2 = ["02", "03", "04"]

In [ ]:
prepare_datasets(train_folders=['../data/DIV2K_train_HR'],
                      test_folders=['../data/DIV2K_train_HR'],
                      min_size=int(96*3),
                      output_folder='dataclass/')

In [ ]:
crop_size = 80
scaling_factor = 4
output_format = "[0, 1]"
input_format = "[0, 1]"
trainset_4_0_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format)
testset_4_0_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format, train=False)

scaling_factor = 4
output_format = "[-1, 1]"
input_format = "[0, 1]"
trainset_4_1_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format)
testset_4_1_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format, train=False)

scaling_factor = 2
output_format = "[-1, 1]"
input_format = "[0, 1]"
trainset_2 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format)
testset_2 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format, train=False)

In [ ]:
def test_ssim_score(model, testset, device="cuda:0", output_format="[0, 1]"):
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
    test_ssim = SSIM(data_range=1.0, device="cpu")
    test_ssim.reset()
    model.eval()
    model.to(device)
    i = 0 
    n = 4
    print_output=False
    for (x, y) in testloader:  # [batch_size x 3 x w x h]
        outputs = model(x.to(device))
        if output_format != "[0, 1]":
            outputs = convert_image(outputs, source=output_format, target="[0, 1]")
            y = convert_image(y, source=output_format, target="[0, 1]")
        test_ssim.update((outputs, y.to(device)))
        i+=1
        if print_output and i % n == n-1:
            ssim_score =  test_ssim.compute()
            test_ssim.reset()
            print(f'ssim score: {ssim_score}')

    ssim_score =  test_ssim.compute()
    test_ssim.reset()
    return ssim_score

In [ ]:
def test_psnr_score(model, testset, device="cuda:0", output_format="[0, 1]"):
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
    test_ssim = PSNR(data_range=1.0)
    test_ssim.reset()
    model.to(device)
    i = 0
    for (x, y) in testloader:  # [batch_size x 3 x w x h]
        outputs = model(x.to(device))
        if output_format != "[0, 1]":
            outputs = convert_image(outputs, source=output_format, target="[0, 1]")
            y = convert_image(y, source=output_format, target="[0, 1]")
        test_ssim.update((outputs, y.to(device)))
    ssim_score =  test_ssim.compute()
    test_ssim.reset()
    
    return ssim_score

PNSR

In [ ]:
# psnr_scores = []
# for name, model in zip(names_2, models_2):
#     psnr = test_psnr_score(model, testset=testset_2, output_format="[-1, 1]")
#     print(f"model {name} has psnr of {psnr}")
# # model 02 has psnr of 28.00429515948226
# # model 03 has psnr of 26.48203127708367
# # model 04 has psnr of 25.2988156897758

In [ ]:
# for name, model in zip(names_4_1_1, models_4_1_1):
#     psnr = test_psnr_score(model, testset=testset_4_1_1, output_format="[-1, 1]")
#     print(f"model {name} has psnr of {psnr}")
# # model 06 has psnr of 26.7430370322221
# # model 05 has psnr of 26.623936126717187


In [ ]:
# for name, model in zip(names_4_0_1, models_4_0_1):
#     psnr = test_psnr_score(model, testset=testset_4_0_1, output_format="[0, 1]")
#     print(f"model {name} has psnr of {psnr}")
# # model mse-ssim has psnr of 25.300528074403182
# # model lpips has psnr of 21.18300433101087

# # model mse-ssim has psnr of 24.468000624166926
# # model mse-lpips has psnr of 23.04132645986445

SSIM

In [ ]:
# ssim_scores = []
# for name, model in zip(names_2, models_2):
#     ssim = test_ssim_score(model, testset=testset_2, output_format="[-1, 1]")
#     print(f"model {name} has ssim of {ssim}")
# # model 02 has ssim of 0.7902148982918878
# # model 03 has ssim of 0.7469442279186242
# # model 04 has ssim of 0.722695882466259

In [ ]:
# for name, model in zip(names_4_1_1, models_4_1_1):
#     ssim = test_ssim_score(model, testset=testset_4_1_1, output_format="[-1, 1]")
#     print(f"model {name} has ssim of {ssim}")
# # model 06 has ssim of 0.6911470746297319
# # model 05 has ssim of 0.6890133445089939

In [ ]:
# for name, model in zip(names_4_0_1, models_4_0_1):
#     ssim = test_ssim_score(model, testset=testset_4_0_1, output_format="[0, 1]")
#     print(f"model {name} has ssim of {ssim}")
# # model mse-ssim has ssim of 0.6789199548727454
# # model lpips has ssim of 0.43677345014694735

Images

In [ ]:
crop_size = 384
scaling_factor = 4
output_format = "[0, 1]"
input_format = "[0, 1]"
trainset_4_0_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format)
testset_4_0_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format, train=False)

scaling_factor = 4
output_format = "[-1, 1]"
input_format = "[0, 1]"
trainset_4_1_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format)
testset_4_1_1 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format, train=False)

scaling_factor = 2
output_format = "[-1, 1]"
input_format = "[0, 1]"
trainset_2 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format)
testset_2 = ImagesDataset("dataclass/", crop_size=crop_size, scaling_factor=scaling_factor, lr_format=input_format, hr_format=output_format, train=False)

In [ ]:
def show_images(img):
    img = img 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
def compare_hr_images(model, evalset, device="cuda:0", index=738, output_format="[0, 1]"):
    image_lr, image_hr = evalset[index]
    with torch.no_grad():
        ouput_lr = model(image_lr.unsqueeze(0).to(device))
        if output_format != "[0, 1]":
            ouput_lr = convert_image(ouput_lr, output_format, "[0, 1]")
            image_hr = convert_image(image_hr, output_format, "[0, 1]")
    show_images(torchvision.utils.make_grid([image_hr, ouput_lr.cpu().detach()[0]]))

In [ ]:
def save_res_images(model, evalset, device="cuda:0", index=738, output_format="[0, 1]", model_name="", name=""):
    image_lr, image_hr = evalset[index]
    with torch.no_grad():
        ouput_lr = model(image_lr.unsqueeze(0).to(device))
        if output_format != "[0, 1]":
            ouput_lr = convert_image(ouput_lr, output_format, "[0, 1]")
            image_hr = convert_image(image_hr, output_format, "[0, 1]")
    save_image(image_hr, "figures/results/"+name+ "_real.png")
    save_image(ouput_lr, "figures/results/" + model_name + "_" +name+"_fake.png")

In [ ]:
# compare_hr_images(models_4_1_1[1], testset_4_1_1, device="cuda:0", index=5, output_format="[-1, 1]")

In [ ]:
# image_name = "tiger_face"
# for name, model in zip(names_2, models_2):
#     save_res_images(model, testset_2, output_format="[-1, 1]", model_name=name, name=image_name)

# for name, model in zip(names_4_1_1, models_4_1_1):
#     save_res_images(model, testset_4_1_1, output_format="[-1, 1]", model_name=name, name=image_name)
# for name, model in zip(names_4_0_1, models_4_0_1):
#     save_res_images(model, testset_4_0_1, output_format="[0, 1]", model_name=name, name=image_name)

In [ ]:
# image_name = "books"
# index=5
# for name, model in zip(names_2, models_2):
#     save_res_images(model, testset_2, output_format="[-1, 1]", model_name=name, name=image_name, index=index)
# for name, model in zip(names_4_1_1, models_4_1_1):
#     save_res_images(model, testset_4_1_1, output_format="[-1, 1]", model_name=name, name=image_name, index=index)
# for name, model in zip(names_4_0_1, models_4_0_1):
#     save_res_images(model, testset_4_0_1, output_format="[0, 1]", model_name=name, name=image_name, index=index)

In [ ]:
# image_name = "buildings"
# index=93
# for name, model in zip(names_2, models_2):
#     save_res_images(model, testset_2, output_format="[-1, 1]", model_name=name, name=image_name, index=index)
# for name, model in zip(names_4_1_1, models_4_1_1):
#     save_res_images(model, testset_4_1_1, output_format="[-1, 1]", model_name=name, name=image_name, index=index)
# for name, model in zip(names_4_0_1, models_4_0_1):
#     save_res_images(model, testset_4_0_1, output_format="[0, 1]", model_name=name, name=image_name, index=index)

In [ ]:
# image_name = "butterfly"
# index=5
# for name, model in zip(names_2, models_2):
#     save_res_images(model, testset_2, output_format="[-1, 1]", model_name=name, name=image_name, index=index)
# for name, model in zip(names_4_1_1, models_4_1_1):
#     save_res_images(model, testset_4_1_1, output_format="[-1, 1]", model_name=name, name=image_name, index=index)
# for name, model in zip(names_4_0_1, models_4_0_1):
#     save_res_images(model, testset_4_0_1, output_format="[0, 1]", model_name=name, name=image_name, index=index)

In [ ]:
# name="starfish"
# save_res_images(models_4_0_1[0], testset_4_0_1, output_format="[0, 1]", model_name="mse-ssim", name=name, index=0)

In [ ]:
# save_res_images(models_4_1_1[0], testset_4_1_1, output_format="[-1, 1]", model_name="06", name=name, index=0)

In [ ]:
# save_res_images(models_4_0_1[1], testset_4_0_1, output_format="[0, 1]", model_name="mse-lpips", name=name, index=0)

Archive

In [ ]:
# if scaling_factor == 4 and output_format == "[-1, 1]":
#     gen = load_gen_05_05_2021()
#     ssim_score = test_ssim_score(gen, testset=testset, device="cuda:0")
#     print(f"model gen_05_05_21 has ssim {ssim_score}")
#     psnr = test_psnr_score(gen, testset=testset)
#     print(f"model gen_05_05_21 has psnr {psnr}")
# elif scaling_factor == 4 and output_format == "[0, 1]":
#     gen = load_gen_11_05_2021()
#     ssim_score = test_ssim_score(gen, testset=testset, device="cuda:0")
#     print(f"model gen_11_05_21 has ssim {ssim_score}")
#     psnr = test_psnr_score(gen, testset=testset)
#     print(f"model gen_11_05_21 has psnr {psnr}")

# model gen_11_05_21 has ssim 0.6303913038208995
# model gen_11_05_21 has psnr 26.780872012435065

    

In [ ]:
# 11_05_21 -> 0.6256
# 05-05-21 -> 0.0077

In [ ]:
# upscale 2 : pnsr
# if scaling_factor == 2 and output_format == "[-1, 1]":
#     models = [load_gen_02_05_2021(), load_gen_03_05_2021()]
#     names = ["gen_02_05_2021", "gen_03_05_2021"]
#     psnr_scores = []
#     for name, model in zip(names, models):
#         psnr = test_psnr_score(model, testset=testset)
#         print(f"model {name} has psnr of {psnr}")


In [ ]:
# upscale 2 : ssim
# if scaling_factor == 2 and output_format == "[-1, 1]":
#     models = [load_gen_02_05_2021(), load_gen_03_05_2021()]
#     names = ["gen_02_05_2021", "gen_03_05_2021"]
#     psnr_scores = []
#     for name, model in zip(names, models):
#         ssim = test_ssim_score(model, testset=testset)
#         print(f"model {name} has ssim of {ssim}")